# Juntando as Peças 🧩

## HTTP(S) 🕸️ e Sistema de Arquivos 🗂️

### Georreferenciamento

<pre>Vamos baixar os dados de georreferenciamento de estados e municípios brasileiros para o nosso sistema de arquivos... nós os utilizaremos no decorrer das aulas!</pre>

👉 dados de <a href='https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/estados.csv'>georreferenciamento dos estados brasileiros</a>

👉 dados de <a href='https://raw.githubusercontent.com/kelvins/Municipios-Brasileiros/main/csv/municipios.csv'>georreferenciamento dos municípios brasileiros</a>

#### Funções Auxiliares

In [1]:
import os, requests

def build_path(subfolder = 'raw'):
    current_folder = os.getcwd()
    folderpath = os.path.join(current_folder, os.pardir, 'project', 'data', subfolder)
    folderpath = os.path.abspath(folderpath)
    if not os.path.exists(folderpath): os.makedirs(folderpath)
    return folderpath

def read_file(url):
    response = requests.get(url)

    if response.status_code == 200:
        content = response.content
        return content

def download_file_and_save(url, filename):
    folderpath = build_path()
    filepath = os.path.join(folderpath, filename)
    
    content = read_file(url)

    if not os.path.exists(folderpath):
        os.makedirs(folderpath)

    with open(filepath, mode='wb') as file:
        file.write(content)

#### Código

In [2]:
download_file_and_save(
    'https://raw.githubusercontent.com/'+
    'kelvins/Municipios-Brasileiros/main/csv/estados.csv',
    'estados_georreferenciamento.csv')
download_file_and_save(
    'https://raw.githubusercontent.com/'+
    'kelvins/Municipios-Brasileiros/main/csv/municipios.csv',
    'municipios_georreferenciamento.csv')

### Código IBGE de Estado

👉 Lista de <a href='https://servicodados.ibge.gov.br/api/docs/'>API do IBGE</a>

<pre>Vamos baixar os dados de códigos IBGE dos estados brasileiros... nós os utilizaremos no decorrer das aulas!</pre>

#### Funções Auxiliares

In [3]:
import json

"""
workaround para contornar a exceção: \
    SSLError: [SSL: UNSAFE_LEGACY_RENEGOTIATION_DISABLED] \
    unsafe legacy renegotiation disabled (_ssl.c:997)

é possível considerando que estamos tratando apenas de dados públicos
"""

import requests, urllib3, ssl

class CustomHttpAdapter (requests.adapters.HTTPAdapter):
    def __init__(self, ssl_context=None, **kwargs):
        self.ssl_context = ssl_context
        super().__init__(**kwargs)
    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = urllib3.poolmanager.PoolManager(
            num_pools=connections, maxsize=maxsize,
            block=block, ssl_context=self.ssl_context)

def get_legacy_session():
    ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
    ctx.options |= 0x4  # OP_LEGACY_SERVER_CONNECT
    session = requests.session()
    session.mount('https://', CustomHttpAdapter(ctx))
    return session

def access_api_and_save(url, filename):
    response = get_legacy_session().get(url)

    if response.status_code == 200:
        content = response.json()
        
        folderpath = build_path()
        filepath = os.path.join(folderpath, filename)
        with open(filepath, mode='w') as jsonfile:
            json.dump(content, jsonfile)

#### Código

In [4]:
access_api_and_save('https://servicodados.ibge.gov.br/'+
                    'api/v1/localidades/estados', 
                    'estados_codigos.json')

## Aspectos de Banco de Dados 🏦 🎲

### Código IBGE de Município

<pre>Vamos baixar os scripts de banco de dados para criar e popular a tabela de códigos IBGE dos municípios brasileiros... nós os utilizaremos no decorrer das aulas!</pre>

👉 script de banco de dados para <a href='https://raw.githubusercontent.com/andrebaltieri/ibge/main/SQLite/Schema.sql'>criar a estrutura da tabela</a> de código IBGE do Município

👉 script de banco de dados para <a href='https://raw.githubusercontent.com/andrebaltieri/ibge/main/SQLite/Data.sql'>popular a tabela</a> de código IBGE do Município

#### Função Auxiliar

In [5]:
import sqlite3

def execute_script(db_filename, sql_filename, delete_script=False):
    folderpath = build_path()
    dbpath = os.path.join(folderpath, db_filename)
    sqlpath = os.path.join(folderpath, sql_filename)
    
    with sqlite3.connect(dbpath) as conn:
        cursor = conn.cursor()

    with open(sqlpath, encoding='utf8') as sqlfile:
        script = sqlfile.read()

    cursor = cursor.executescript(script)

    conn.commit()
    conn.close()
    
    if delete_script: os.remove(sqlpath)

#### Código

In [6]:
download_file_and_save('https://raw.githubusercontent.com/'+
                       'andrebaltieri/ibge/main/SQLite/Schema.sql',
                       'schema.sql')
download_file_and_save('https://raw.githubusercontent.com/'+
                       'andrebaltieri/ibge/main/SQLite/Data.sql',
                       'data.sql')

execute_script('municipios_codigos.db', 'schema.sql', 
               delete_script=True)
execute_script('municipios_codigos.db', 'data.sql', 
               delete_script=True)